* This notebook is a modified version of *Sample Architecture* to try out So2Sat LCZ42 Data Set

In [1]:
import logging
import sys

from naslib.defaults.trainer import Trainer

from naslib.search_spaces import (
    DartsSearchSpace,
    SimpleCellSearchSpace,
    NasBench101SearchSpace,
    HierarchicalSearchSpace,
)
from naslib.search_spaces.nasbench101 import graph

from naslib.utils import get_dataset_api, setup_logger, utils

import numpy as np

device: cpu
device: cpu
device: cpu
device: cpu
device: cpu
device: cpu


### Defining the Dataset

In [2]:
config = utils.get_config_from_args(config_type="nas")

In [3]:
dataset_api = get_dataset_api(config.search_space, config.dataset)

Loading dataset from file... This may take a few minutes...
Loaded dataset in 3 seconds


### Defining the Cell Configuration

In [4]:
INPUT = "input"
OUTPUT = "output"
CONV3X3 = "conv3x3-bn-relu"
CONV1X1 = "conv1x1-bn-relu"
MAXPOOL3X3 = "maxpool3x3"
OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]

NUM_VERTICES = 7
OP_SPOTS = NUM_VERTICES - 2
MAX_EDGES = 9

In [5]:
def sample_random_architecture(dataset_api, arch_limit=10):
        """
        This will sample a random architecture and update the edges in the
        naslib object accordingly.
        From the NASBench repository:
        one-hot adjacency matrix
        draw [0,1] for each slot in the adjacency matrix
        """
        architectures = []
        while len(architectures) < arch_limit:
            matrix = np.random.choice([0, 1], size=(NUM_VERTICES, NUM_VERTICES))
            matrix = np.triu(matrix, 1)
            ops = np.random.choice(OPS, size=NUM_VERTICES).tolist()
            ops[0] = INPUT
            ops[-1] = OUTPUT
            spec = dataset_api["api"].ModelSpec(matrix=matrix, ops=ops)
            if dataset_api["nb101_data"].is_valid(spec):
                architectures.append({"matrix": matrix, "ops": ops})
                #break
        
        return architectures
            
        #self.set_spec({"matrix": matrix, "ops": ops})

### Sampling the architectures

In [6]:
sampled_architectures = sample_random_architecture(dataset_api)

### Converting the architecture into Pytorch Neural Network

In [7]:
from naslib.predictors.utils.models import nasbench1 as nas101_arch
from naslib.predictors.utils.models import nasbench1_spec
import torchvision.transforms as transforms
import torchvision.datasets as dset
from pathlib import Path
import torch.nn.functional as F
from tqdm import tqdm
from time import sleep
import torch

* We are using cifar10 to sample a network, keep in mind that So2Sat LCZ42 **17** classes 
* We need to change the input featurs to **10** for sentinel-2 in **naslib/predictors/utils/models/nasbench1.py** 

In [8]:
spec = nasbench1_spec._ToModelSpec(
    sampled_architectures[0]["matrix"], sampled_architectures[0]["ops"]
)

# change cofar10 output feature to 17
num_classes_dic = {"cifar10": 17, "cifar100": 100, "ImageNet16-120": 120}

network = nas101_arch.Network(
    spec,
    stem_out=128,
    num_stacks=3,
    num_mods=3,
    num_classes=num_classes_dic["cifar10"]
)

### Exporting the Architecture

In [9]:
torch.save(network, './network0')

### Importing the Architecture

In [10]:
network_1 = torch.load('./network0')

### Preparing the network for `So2Sat LCZ42` classification

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [12]:
on_device = network.to(device)

In [13]:
type(on_device)

naslib.predictors.utils.models.nasbench1.Network

* Pre-calculated mean and std values for sentinel-2 data

In [14]:
training_data_mean = [
1.237384229898452759e-01,1.092514395713806152e-01,1.010472476482391357e-01,
1.141963005065917969e-01,1.592123955488204956e-01,1.814149618148803711e-01,
1.745131611824035645e-01,1.949533522129058838e-01,1.542119830846786499e-01,
1.089953780174255371e-01]
training_data_std = [
3.955886885523796082e-02,4.774657264351844788e-02,6.632962822914123535e-02,
6.356767565011978149e-02,7.745764404535293579e-02,9.104172885417938232e-02,
9.217569977045059204e-02,1.016793847084045410e-01,9.986902773380279541e-02,
8.776713907718658447e-02
]
validation_data_mean = [
1.289583146572113037e-01,1.164120137691497803e-01,1.122049614787101746e-01,
1.240097358822822571e-01,1.646174490451812744e-01,1.862829625606536865e-01,
1.791501641273498535e-01,2.004020363092422485e-01,1.738285273313522339e-01,
1.277212053537368774e-01]
validation_data_std = [
3.845561295747756958e-02,4.351711273193359375e-02,5.868862569332122803e-02,
5.489562824368476868e-02,6.446107476949691772e-02,7.579671591520309448e-02,
7.847409695386886597e-02,8.553111553192138672e-02,9.418702870607376099e-02,
8.428058028221130371e-02
]

In [15]:
train_transform = transforms.Compose(
    [
        #transforms.RandomCrop(32, padding=4),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(training_data_mean, training_data_std),
    ]
)

valid_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(validation_data_mean, validation_data_std),
    ]
)


* To be able to use custom data loader, EODataLoader, we need to specify the path of file

In [16]:
from os import path
import sys
sys.path.append(path.abspath('../../../repository/tum-dlr-automl-for-eo/src/tum_dlr_automl_for_eo/datamodules'))
from EODataLoader import EODataModule

* Data set should be under a file named So2SatLCZ, if this path does not exist it will download the data in the following step 

In [17]:
data_module = EODataModule('/Users/safayilmaz/Desktop/DI LAB', 'Sentinel-2')

* This step will prepare the data, namely download it if it does not exist

In [18]:
data_module.prepare_data()

2022-11-30 09:50:31,704 - Data preparation is done, next step is data set-up


* **setup_training_data, setup_validation_data, and setup_testing_data** will read the data from downloaded paths, and apply specified transformations.

In [19]:
data_module.setup_training_data(train_transform)
batch_size = 64
training_data = data_module.training_dataLoader(batch_size = batch_size)

2022-11-30 09:50:33,448 - Number of training data: 352366
2022-11-30 09:50:33,449 - Number of classes: 17


In [20]:
data_module.setup_validation_data(valid_transform)
validation_data = data_module.validation_dataLoader(batch_size = batch_size)

2022-11-30 09:50:36,547 - Number of validation data: 24119
2022-11-30 09:50:36,548 - Number of classes: 17


In [21]:
loss_fn = F.cross_entropy
lr = 0.4
optimizer = torch.optim.SGD(network.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
network.train()
for epoch in range(1, 5):
    with tqdm(training_data, unit="batch") as tepoch:
        for data, target in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = network(data.float())
            predictions = output.argmax(dim=1, keepdim=True).squeeze()
            loss = F.nll_loss(output, target)
            correct = (predictions == target).sum().item()
            accuracy = correct / batch_size
            
            loss.backward()
            optimizer.step()

            tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)
            sleep(0.1)

Epoch 1:   7%| | 393/5506 [26:19<5:41:36,  4.01s/batch, accuracy=1.56, loss=nan]